In [1]:
import docx
import os
import xml.etree.ElementTree as ET
import re



In [2]:
rtf_path = os.path.abspath('sample_rtf')
xml_path = os.path.abspath('sample_xml')
for root, dirs, files in os.walk(rtf_path):
        for file in files:
            print(os.path.join(root, file))

e:\BNY Mellon capstone project\BKG\data\sample_rtf\Northern Trust Corporation, Q1 2020 Earnings Call, Apr 21, 2020.rtf
e:\BNY Mellon capstone project\BKG\data\sample_rtf\The Bank of New York Mellon Corporation, Q2 2023 Earnings Call, Jul 18, 2023 (1).rtf
e:\BNY Mellon capstone project\BKG\data\sample_rtf\The Bank of New York Mellon Corporation, Q3 2023 Earnings Call, Oct 17, 2023 (1).rtf
e:\BNY Mellon capstone project\BKG\data\sample_rtf\The Bank of New York Mellon Corporation, Q4 2023 Earnings Call, Jan 12, 2024 (1).rtf


In [3]:

filename = "Northern Trust Corporation, Q1 2020 Earnings Call, Apr 21, 2020"

In [4]:
import aspose.words as aw
doc = aw.Document(os.path.join(rtf_path, filename+".rtf"))
doc.save(filename+".docx")

In [5]:

doc = docx.Document(filename+".docx")

In [6]:
filename.split(",")[0]

'Northern Trust Corporation'

In [7]:
def remove_empty_columns(arr):
    transposed = list(zip(*arr))

    filtered = [col for col in transposed if any(cell != "" for cell in col)]

    return list(zip(*filtered))

In [8]:
def build_first_table(data):
    data = [list(dict.fromkeys(row)) for row in data]
    data = remove_empty_columns(data)
    root = ET.Element("table",attrib={"id":"0", "name":"Earnings Estimates Comparison Table"})

    time_periods = ET.SubElement(root, "timePeriods")
    for period in data[0][1:]:
        ET.SubElement(time_periods, "period", name=period.replace("-", ""))

    metrics = ET.SubElement(root, "metrics")
    for metric_data in data[2:]:
        metric = ET.SubElement(metrics, "metric", name=metric_data[0])
        for i, value in enumerate(metric_data[1:]):
            period_name = data[0][i + 1]
            value_type = data[1][i + 1]
            ET.SubElement(metric, "value", period=period_name, type=value_type).text = value
    
    return root

def build_second_table(data):
    # clean duplicates   
    data = [list(dict.fromkeys(row)) for row in data]
    data = remove_empty_columns(data)
    
    root = ET.Element("table",attrib={"id":"1","name":"EPS Normalized Comparison Table"})
    

    type_header = data[1]

    time_periods = ET.SubElement(root, "timePeriods") 
    metrics = ET.SubElement(root, "metrics") 
    metric = ET.SubElement(metrics, "metric", name="EPS Normalized")
    for row in data[2:]:
        period = row[0]
        ET.SubElement(time_periods, "period", name=period)
        for i, cell in enumerate(row[1:], 1):
            type_ = type_header[i]  
            if type_ in ["CONSENSUS", "ACTUAL", "SURPRISE"]:
                value_element = ET.SubElement(metric, "value")
                value_element.text = cell
                value_element.set("period", period)
                value_element.set("type", type_)

    return root

def build_third_table(data,company):
    id = 0
    root = ET.Element("CallParticipants")
    speaker_list = {}
         
    current_group = ''
    for row in data[1:]:
        
        row_data = '\n \n \n'.join(row).strip()
        elements = row_data.split('\n \n \n')
        for element in elements:

            lines = element.split('\n')

            if len(lines) == 1 :
                current_group = lines[0].strip()
                
            if len(lines) > 1:

                name = lines[0].strip()
                
                position = lines[1].strip()
                if current_group == "EXECUTIVES":
                    position = company +", " + position
                person_element = ET.SubElement(root, "person", position=position, group=current_group, id = str(id))
                person_element.text = name
                speaker_list[name] = str(id)
                id+=1
    return root, speaker_list


def process_presentation(dialog,speaker_list, name):
    paragraph = dialog.split('\n')

    conversation = ET.Element("section", attrib={"name": name})
    i = 0 
    while i < len(paragraph):
        if paragraph[i].strip() in speaker_list:
            id = speaker_list[paragraph[i].strip()]
            position = paragraph[i+1].strip()
            statement = ET.SubElement(conversation, "statement")
            speaker_element = ET.SubElement(statement, "speaker", id=id, position=position)
            speaker_element.text = paragraph[i].strip()
            text = ""
            para = ET.SubElement(speaker_element, "text")
            i += 2
            while i < len(paragraph) and paragraph[i].strip() not in speaker_list and paragraph[i].strip()!= "Operator":
                if len(paragraph[i].strip()) != 0:
                    text += paragraph[i] + "\n"
                i += 1
            para.text = text.strip()
            
        elif "Operator" in paragraph[i]:
            id = "-1"
            position = ""
            statement = ET.SubElement(conversation, "statement")
            speaker_element = ET.SubElement(statement, "speaker", id=id, position=position)
            speaker_element.text = "Operator"
            text = ""
            para = ET.SubElement(speaker_element, "text")
            i += 1
            while i < len(paragraph) and paragraph[i].strip() not in speaker_list:
                text += paragraph[i] + "\n"
                i += 1
            para.text = text.strip()
            
        else:
            i += 1
    return conversation


def process_dialog(dialog,speaker_list, name):
    question_id = -1
    end = False
    paragraph = dialog.split('\n')
    cur_question = None
    conversation = ET.Element("section", attrib={"name": name})
    i = 0 
    hasSub = False
    while i < len(paragraph):
        # print(paragraph[i])
        # print("------------------------")
        if paragraph[i].strip() in speaker_list:
            id = speaker_list[paragraph[i].strip()]
            position = paragraph[i+1].strip()
            if end:
                context = ET.SubElement(conversation, "ending", id = str(question_id))
            elif cur_question == None:
                context = ET.SubElement(conversation, "question", id = str(question_id))
                cur_question = paragraph[i].strip()
            elif paragraph[i].strip() == cur_question :
                context = ET.SubElement(conversation, "followQuestion")
                hasSub = True
            elif hasSub and paragraph[i].strip()!= cur_question:
                context = ET.SubElement(conversation, "followAnswer")
                hasSub = False
            else:
                context = ET.SubElement(conversation, "answer", id = str(question_id))
            speaker_element = ET.SubElement(context, "speaker", id=id, position=position)
            speaker_element.text = paragraph[i].strip()
            text = ""
            para = ET.SubElement(speaker_element, "text")
            i += 2
            while i < len(paragraph) and paragraph[i].strip() not in speaker_list and paragraph[i].strip()!= "Operator" and not paragraph[i].startswith("Operator"):
                # print(paragraph[i])
                # print(paragraph[i].startswith("Operator"))
                # print("--------------------------")
                if len(paragraph[i].strip()) != 0:
                    text += paragraph[i] + "\n"
                i += 1
            para.text = text.strip()
            
        elif "Operator" in paragraph[i]:
            id = "-1"
            position = ""
            cur_question = None
            hasSub = False
            question_id += 1
            context =ET.SubElement(conversation, "transition") 
            speaker_element = ET.SubElement(context, "speaker", id=id, position=position)
            speaker_element.text = "Operator"
            text = ""
            para = ET.SubElement(speaker_element, "text")
            paragraph[i] = paragraph[i].replace("Operator", "")
            while i < len(paragraph) and paragraph[i].strip() not in speaker_list:
                text += paragraph[i] + "\n"
                i += 1
            para.text = text.strip()
            if "conclude" in para.text:
                context.tag = "ending"
                end = True
                
            
        else:
            i += 1

    return conversation

In [9]:
def prettify(element, indent='    ', level=0):
    
    if element:  
        if not element.text or not element.text.strip():
            element.text = '\n' + indent * (level + 1)
        if not element.tail or not element.tail.strip():
            element.tail = '\n' + indent * level
    else:
        if level and (not element.tail or not element.tail.strip()):
            element.tail = '\n' + indent * level
    
    for subelement in element:
        prettify(subelement, indent, level + 1)

In [12]:
company = ""
for i, paragraph in enumerate(doc.paragraphs):

    if i ==2 :
        company = paragraph.text
        break
tables = []
for table_index, table in enumerate(doc.tables):
        t = []
        for row_index, row in enumerate(table.rows):
            row_data = []

           
            for cell in row.cells:
                row_data.append(cell.text.strip())

            if all(element == "" for element in row_data):
                 continue
            t.append(row_data)
        if t== [['']] or t ==[]:
             continue
        tables.append(t)
        
t1 = build_first_table(tables[0])
t2 = build_second_table(tables[1])
t3,speaker_list = build_third_table(tables[3],company)
sec1 = ET.Element("section", attrib={"name": "financial tables"})
sec1.append(t1)
sec1.append(t2)
t3.tag = "section"
t3.set("name", "call participants")

In [13]:
body = ET.Element("body")
company = ""
title = ""
time = ""
currency = ""
note = ""
QA = None
presentation = None
for i, paragraph in enumerate(doc.paragraphs):

    if i ==2 :
        company = paragraph.text
        
    elif i == 3:
        title = paragraph.text
    elif i == 4:
        time = paragraph.text
    elif i == 6:
        currency= paragraph.text
    elif i == 7:
        note= paragraph.text
    
    elif paragraph.text.strip().startswith("Question and Answer"):
        QA = process_dialog(paragraph.text,speaker_list,"Question and Answer")
    elif paragraph.text.strip().startswith("Presentation"):
        
        presentation = process_presentation(paragraph.text,speaker_list,"Presentation ")
    
header = ET.Element("header")

ET.SubElement(header, "company").text = company
ET.SubElement(header, "title").text = title
ET.SubElement(header, "time").text = time
ET.SubElement(header, "currency").text = currency
ET.SubElement(header, "note").text = note



In [14]:
body.append(sec1)
body.append(t3)
body.append(presentation)
body.append(QA)
root = ET.Element("Transcript")
root.append(header)
root.append(body)

In [15]:
# root = ET.Element("body")
# sec1 = ET.Element("section", attrib={"name": "financial tables"})
# t1 = build_first_table(tables[0])
# t2 = build_second_table(tables[1])
# t3,speaker_list = build_third_table(tables[3])
# sec1.append(t1)
# sec1.append(t2)
# t3.tag = "section"
# t3.set("name", "call participants")
# root.append(header)
# root.append(sec1)
# root.append(t3)


prettify(root)


tree = ET.ElementTree(root)
tree.write(os.path.join(xml_path,filename+".xml"), encoding="utf-8", xml_declaration=True)

In [16]:
os.remove(filename+'.docx')

## get images

In [71]:
import zipfile
import os
from PIL import Image
from io import BytesIO

def extract_and_open_images(docx_filename):

    temp_dir = "extracted_images"
    os.makedirs(temp_dir, exist_ok=True)

    with zipfile.ZipFile(docx_filename, 'r') as docx:

        for file in docx.namelist():
            if file.startswith('word/media/'):
                image_data = docx.read(file)
                image = Image.open(BytesIO(image_data))
                image.show()  


                image_filename = os.path.join(temp_dir, os.path.basename(file))
                image.save(image_filename)


docx_filename = filename+'.docx'  
extract_and_open_images(docx_filename)